Generate synthetic data based on [flights dataset](https://www.kaggle.com/datasets/eliasdabbas/search-engine-results-flights-tickets-keywords)

In [ ]:
import pandas as pd
import openai

In [ ]:
PAIRS_NUM = 5
MAX_TOKENS = 500
TEMPERATURE = 0.9


OPENAI_API_KEY = '<yor key>'
openai.api_key = OPENAI_API_KEY

In [ ]:
def create_pairs(doc):
    text = openai.Completion.create(
            model='text-davinci-003',
            prompt=f'Generate numerated list of {PAIRS_NUM} pairs of right \
            search queries for text. Queries in pair should be similar by meaning and separated by ":". \
            Text: {doc}\nList:',
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE

        )
    pairs = []
    generated_pairs = text['choices'][0]['text']
    for el in generated_pairs.split('\n'):
        if el=='':
            continue
        else:
            if el[0].isdigit():
                splitted = el[3:].split(':')

                to_add = []

                if len(splitted) == 1:
                    splitted.append('')
                elif len(splitted)==0:
                    splitted.append('')
                    splitted.append('')

                pairs.append([splitted[0].strip(), splitted[1].strip()])

    return pairs


def create_dataset(data, text_file):
    to_csv_samples = {'first': [], 'second' : []}
    for i, ind in enumerate(data.index.tolist()):
        pairs = create_pairs(data.loc[ind]['snippet'])

        for pair in pairs:


            to_csv_samples['first'].append(pair[0])
            to_csv_samples['second'].append(pair[1])

            text_file.write(f'{ind} : {pair[0]} : {pair[1]}\n')

        print(f'created pairs for doc {i+1} / {len(data)}\n')



Download dataset from [source](https://www.kaggle.com/datasets/eliasdabbas/search-engine-results-flights-tickets-keywords)


In [ ]:
flights = pd.read_csv('flights.csv')
flights_ranks = flights[['searchTerms', 'rank', 'snippet']]

In [ ]:
with open('data/samples.txt', 'a') as f:
    create_dataset(flights_ranks, f)